In [242]:
#Funcion para lectura de datos .gpx
import gpxpy
import pandas as pd
import sqlite3
from geopy import distance

def read_gpx(file: str) -> pd.DataFrame:
    df = None
    points = []
    with open(file) as f:
        gpx = gpxpy.parse(f)

    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time':p.time,
                'latitude':p.latitude,
                'longitude':p.longitude,
                'elevation':p.elevation
            })
    df = pd.DataFrame.from_records(points)
    
    return df

In [243]:
#Datos
df1 = read_gpx('recovery.01-Mar-2022-1533.gpx')
df2 = read_gpx('recovery.05-Mar-2022.1025.gpx')
df3 = read_gpx('recovery.25-May-2022-0907.gpx')

In [244]:
#Copias a los datos originales
Data1 = df1.copy()
Data2 = df2.copy()
Data3 = df3.copy()

In [245]:
Data1

,time,latitude,longitude,elevation
0,2022-03-01 20:33:48+00:00,6.297476,-75.578192,1668.879307
1,2022-03-01 20:33:49+00:00,6.297476,-75.578146,1668.386143
2,2022-03-01 20:33:57+00:00,6.297460,-75.578169,1667.904453


In [246]:
#Validar si hay datos nulos
Data2.isnull().any()

time         False
latitude     False
longitude    False
elevation    False
dtype: bool

In [248]:
#Quitar duplicados por la columna de fecha 
Data1.drop_duplicates(subset='time', keep='first', inplace=True)
Data2.drop_duplicates(subset='time', keep='first', inplace=True)
Data3.drop_duplicates(subset='time', keep='first', inplace=True)

In [249]:
#Separar Campos de fecha y hora
def separar_campos_fechas(df):
    df["Date_Time"] = pd.to_datetime(df["time"], format="%y-%m-%d").dt.tz_localize(None)
    df['date'] = pd.to_datetime(df['Date_Time'].dt.date)
    df['time'] = df['Date_Time'].dt.time
    df['year'] = df['Date_Time'].dt.year
    df['month'] = df['Date_Time'].dt.month
    df['day'] = df['Date_Time'].dt.day
    df['hour'] = df['Date_Time'].dt.hour
    df['minutes'] = df['Date_Time'].dt.minute
    df['seconds'] = df['Date_Time'].dt.second
    return df

In [250]:
# Calculo de nuevas columnas a los DataFrame
separar_campos_fechas(Data1)
separar_campos_fechas(Data2)
separar_campos_fechas(Data3)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds
0,21:24:46.433000,6.208559,-75.571630,1558.78,2022-05-25 21:24:46.433,2022-05-25,2022,5,25,21,24,46
1,14:05:32.284000,6.268901,-75.593216,1548.51,2022-05-26 14:05:32.284,2022-05-26,2022,5,26,14,5,32
2,14:05:34,6.268870,-75.593087,1551.91,2022-05-26 14:05:34.000,2022-05-26,2022,5,26,14,5,34
3,14:05:36,6.268792,-75.593007,1551.53,2022-05-26 14:05:36.000,2022-05-26,2022,5,26,14,5,36
4,14:05:37,6.268722,-75.592900,1551.37,2022-05-26 14:05:37.000,2022-05-26,2022,5,26,14,5,37
...,...,...,...,...,...,...,...,...,...,...,...,...
653,14:32:21,6.208590,-75.571338,1553.61,2022-05-26 14:32:21.000,2022-05-26,2022,5,26,14,32,21
654,14:32:27,6.208485,-75.571341,1548.82,2022-05-26 14:32:27.000,2022-05-26,2022,5,26,14,32,27
655,14:32:34,6.208457,-75.571435,1549.02,2022-05-26 14:32:34.000,2022-05-26,2022,5,26,14,32,34
656,14:32:41,6.208468,-75.571525,1559.75,2022-05-26 14:32:41.000,2022-05-26,2022,5,26,14,32,41


In [251]:
#Quitar la fila 1 del dataframe 3, ya que corresponde a un dato atípico
Data3 = Data3.drop(index=0).reset_index()

In [252]:
#Calcular la distancia entre puntos (distancia euclidiana en grados decimales)
def distancia_puntos(df):
    distances = [0]
    for i in range(df.shape[0]-1):
        Coordenada1 = (df['latitude'].iloc[i], df['longitude'].iloc[i])
        Coordenada2 = (df['latitude'].iloc[i+1], df['longitude'].iloc[i+1])
        distancia = distance.distance(Coordenada1, Coordenada2).m
        distances.append(distancia)

    df['distance'] = distances
    return df

In [253]:
distancia_puntos(Data1)
distancia_puntos(Data2)
distancia_puntos(Data3)

,index,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance
0,1,14:05:32.284000,6.268901,-75.593216,1548.51,2022-05-26 14:05:32.284,2022-05-26,2022,5,26,14,5,32,0.000000
1,2,14:05:34,6.268870,-75.593087,1551.91,2022-05-26 14:05:34.000,2022-05-26,2022,5,26,14,5,34,14.781267
2,3,14:05:36,6.268792,-75.593007,1551.53,2022-05-26 14:05:36.000,2022-05-26,2022,5,26,14,5,36,12.350126
3,4,14:05:37,6.268722,-75.592900,1551.37,2022-05-26 14:05:37.000,2022-05-26,2022,5,26,14,5,37,14.100135
4,5,14:05:38,6.268672,-75.592772,1551.04,2022-05-26 14:05:38.000,2022-05-26,2022,5,26,14,5,38,15.141865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,653,14:32:21,6.208590,-75.571338,1553.61,2022-05-26 14:32:21.000,2022-05-26,2022,5,26,14,32,21,10.354379
653,654,14:32:27,6.208485,-75.571341,1548.82,2022-05-26 14:32:27.000,2022-05-26,2022,5,26,14,32,27,11.715680
654,655,14:32:34,6.208457,-75.571435,1549.02,2022-05-26 14:32:34.000,2022-05-26,2022,5,26,14,32,34,10.828659
655,656,14:32:41,6.208468,-75.571525,1559.75,2022-05-26 14:32:41.000,2022-05-26,2022,5,26,14,32,41,10.002506


In [254]:
#Calcular la velocidad entre puntos (Velocidad en grados decimales / segundos)
def velocidad_puntos(df):
    tiempo = pd.to_datetime(df['Date_Time'])
    time_delta = tiempo.diff().astype('timedelta64[s]')
    df['time_delta'] = time_delta
    speed = df['distance'] / df['time_delta']
    df['speed'] = speed
    return df

In [255]:
velocidad_puntos(Data1)
velocidad_puntos(Data2)
velocidad_puntos(Data3)

,index,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,time_delta,speed
0,1,14:05:32.284000,6.268901,-75.593216,1548.51,2022-05-26 14:05:32.284,2022-05-26,2022,5,26,14,5,32,0.000000,NaN,NaN
1,2,14:05:34,6.268870,-75.593087,1551.91,2022-05-26 14:05:34.000,2022-05-26,2022,5,26,14,5,34,14.781267,1.0,14.781267
2,3,14:05:36,6.268792,-75.593007,1551.53,2022-05-26 14:05:36.000,2022-05-26,2022,5,26,14,5,36,12.350126,2.0,6.175063
3,4,14:05:37,6.268722,-75.592900,1551.37,2022-05-26 14:05:37.000,2022-05-26,2022,5,26,14,5,37,14.100135,1.0,14.100135
4,5,14:05:38,6.268672,-75.592772,1551.04,2022-05-26 14:05:38.000,2022-05-26,2022,5,26,14,5,38,15.141865,1.0,15.141865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,653,14:32:21,6.208590,-75.571338,1553.61,2022-05-26 14:32:21.000,2022-05-26,2022,5,26,14,32,21,10.354379,6.0,1.725730
653,654,14:32:27,6.208485,-75.571341,1548.82,2022-05-26 14:32:27.000,2022-05-26,2022,5,26,14,32,27,11.715680,6.0,1.952613
654,655,14:32:34,6.208457,-75.571435,1549.02,2022-05-26 14:32:34.000,2022-05-26,2022,5,26,14,32,34,10.828659,7.0,1.546951
655,656,14:32:41,6.208468,-75.571525,1559.75,2022-05-26 14:32:41.000,2022-05-26,2022,5,26,14,32,41,10.002506,7.0,1.428929


In [256]:
#Poner primer registro del dataframe resultado en cero
Data1['time_delta'][0] = 0  
Data1['speed'][0] = 0
Data2['time_delta'][0] = 0
Data2['speed'][0] = 0
Data3['time_delta'][0] = 0
Data3['speed'][0] = 0

C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_8920\3986897298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data1['time_delta'][0] = 0
C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_8920\3986897298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data1['speed'][0] = 0
C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_8920\3986897298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data2['time_delta'][0] = 0
C:\Users\CCOSORIO\

In [257]:
def aceleracion_punto(df):
    aceleracion = df['speed'].diff() / df['time_delta']
    df["acceleration"] = aceleracion
    return df


In [258]:
aceleracion_punto(Data1)
aceleracion_punto(Data2)
aceleracion_punto(Data3)

,index,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,time_delta,speed,acceleration
0,1,14:05:32.284000,6.268901,-75.593216,1548.51,2022-05-26 14:05:32.284,2022-05-26,2022,5,26,14,5,32,0.000000,0.0,0.000000,NaN
1,2,14:05:34,6.268870,-75.593087,1551.91,2022-05-26 14:05:34.000,2022-05-26,2022,5,26,14,5,34,14.781267,1.0,14.781267,14.781267
2,3,14:05:36,6.268792,-75.593007,1551.53,2022-05-26 14:05:36.000,2022-05-26,2022,5,26,14,5,36,12.350126,2.0,6.175063,-4.303102
3,4,14:05:37,6.268722,-75.592900,1551.37,2022-05-26 14:05:37.000,2022-05-26,2022,5,26,14,5,37,14.100135,1.0,14.100135,7.925072
4,5,14:05:38,6.268672,-75.592772,1551.04,2022-05-26 14:05:38.000,2022-05-26,2022,5,26,14,5,38,15.141865,1.0,15.141865,1.041730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,653,14:32:21,6.208590,-75.571338,1553.61,2022-05-26 14:32:21.000,2022-05-26,2022,5,26,14,32,21,10.354379,6.0,1.725730,-0.013748
653,654,14:32:27,6.208485,-75.571341,1548.82,2022-05-26 14:32:27.000,2022-05-26,2022,5,26,14,32,27,11.715680,6.0,1.952613,0.037814
654,655,14:32:34,6.208457,-75.571435,1549.02,2022-05-26 14:32:34.000,2022-05-26,2022,5,26,14,32,34,10.828659,7.0,1.546951,-0.057952
655,656,14:32:41,6.208468,-75.571525,1559.75,2022-05-26 14:32:41.000,2022-05-26,2022,5,26,14,32,41,10.002506,7.0,1.428929,-0.016860


In [259]:
#Poner primer registro del dataframe resultado en cero
Data1['acceleration'][0] = 0  
Data2['acceleration'][0] = 0
Data3['acceleration'][0] = 0

C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_8920\369125268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data1['acceleration'][0] = 0
C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_8920\369125268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data2['acceleration'][0] = 0
C:\Users\CCOSORIO\AppData\Local\Temp\ipykernel_8920\369125268.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data3['acceleration'][0] = 0


In [260]:
#Cargar datos a sqlite
def ingestar_sqlite(df, BD = 'Base_Coordenadas.db', Tabla = 'Datos'):
    conn = sqlite3.connect(BD)
    df.to_sql(Tabla, conn, if_exists='replace', index = False)
    df_consulta = pd.read_sql(f'SELECT * FROM {Tabla}', conn)
    conn.close()
    return df_consulta

In [261]:
ingestar_sqlite(Data1, BD = 'Base_Coordenadas1.db', Tabla = 'Datos_1')
ingestar_sqlite(Data2, BD = 'Base_Coordenadas2.db', Tabla = 'Datos_2')
ingestar_sqlite(Data3, BD = 'Base_Coordenadas3.db', Tabla = 'Datos_3')

,index,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,time_delta,speed,acceleration
0,1,14:05:32.284000,6.268901,-75.593216,1548.51,2022-05-26 14:05:32.284000,2022-05-26 00:00:00,2022,5,26,14,5,32,0.000000,0.0,0.000000,0.000000
1,2,14:05:34.000000,6.268870,-75.593087,1551.91,2022-05-26 14:05:34,2022-05-26 00:00:00,2022,5,26,14,5,34,14.781267,1.0,14.781267,14.781267
2,3,14:05:36.000000,6.268792,-75.593007,1551.53,2022-05-26 14:05:36,2022-05-26 00:00:00,2022,5,26,14,5,36,12.350126,2.0,6.175063,-4.303102
3,4,14:05:37.000000,6.268722,-75.592900,1551.37,2022-05-26 14:05:37,2022-05-26 00:00:00,2022,5,26,14,5,37,14.100135,1.0,14.100135,7.925072
4,5,14:05:38.000000,6.268672,-75.592772,1551.04,2022-05-26 14:05:38,2022-05-26 00:00:00,2022,5,26,14,5,38,15.141865,1.0,15.141865,1.041730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,653,14:32:21.000000,6.208590,-75.571338,1553.61,2022-05-26 14:32:21,2022-05-26 00:00:00,2022,5,26,14,32,21,10.354379,6.0,1.725730,-0.013748
653,654,14:32:27.000000,6.208485,-75.571341,1548.82,2022-05-26 14:32:27,2022-05-26 00:00:00,2022,5,26,14,32,27,11.715680,6.0,1.952613,0.037814
654,655,14:32:34.000000,6.208457,-75.571435,1549.02,2022-05-26 14:32:34,2022-05-26 00:00:00,2022,5,26,14,32,34,10.828659,7.0,1.546951,-0.057952
655,656,14:32:41.000000,6.208468,-75.571525,1559.75,2022-05-26 14:32:41,2022-05-26 00:00:00,2022,5,26,14,32,41,10.002506,7.0,1.428929,-0.016860


In [262]:
Data3['time_delta'].sum()

1636.0

In [264]:
Data2.to_excel("Datos.xlsx", index=False)